# Topic analysis of covid-19 related literature

## Datasets

The corpus of documents has been retrieved from [LitCovid](https://www.ncbi.nlm.nih.gov/research/coronavirus/), a curated open-resource literature of pubmed research papers related to new coronavirus disease COVID-19, and from [medRxiv and bioRxiv COVID-19 SARS-CoV-2](https://connect.biorxiv.org/relate/content/181) preprints. The text of each document, composed by the concatenation of titles and abstracts, has been tokenized, stemmed, and filtered by stopwords. Duplicate documents, due in some case to edited paper available also on preprint servers, have been removed by comparing the title string with Levenshtein distance.

In [ ]:
import pandas as pd
import json
from RISparser import readris
import dateutil.parser as parser

dois=[]
docs = []
titles = []
year = []
source = []
title_len =[]
with open('data/06202020.litcovid.export.ris', 'r') as bibliography_file:
    entries = readris(bibliography_file)
    for entry in entries:
        dois.append(entry['doi'])
        titles.append(entry['title'])
        docs.append(entry['title'] +' '+ entry['abstract'])
        source.append('pubmed')
        year.append(entry['year'])
        title_len.append(len(entry['title']))

with open('data/biomed-covid.json') as json_file:
    data = json.load(json_file)

for i in data['rels']:
    #if not i['rel_doi'] in dois:
    dois.append(i['rel_doi'])
    titles.append(i['rel_title'])
    docs.append(i['rel_title']+' '+i['rel_abs'])
    source.append(i['rel_site'])
    year.append(str(parser.parse(i['rel_date']).year))
    title_len.append(len(i['rel_title']))

print(len(docs))
df = pd.DataFrame({'year':year,'doi':dois,'title':titles,
                   'txt':docs,'source':source,
                  'tlen':title_len})

df['txt'] = df['txt'].str.lower()

df = df[df['txt'].str.match('\w')]
df = df[(df['year']=='2020') | (df['year']=='2019')]
df = df[df['tlen']<400]

df.shape


## Tokenization, stemming, and stop words removing

The corpus text content, composed by title and abstract, has been processed to extract tokens, i.e. alphanumeric sequence of characters separated by space. Punctuation and pure numbers have been removed. Each word has then been reconducted to its root lemma to normalize the vocabulary and avoid redudant words due to morphological and inflexional endings in English. Finally common english words has been removed.

In [ ]:
# Tokenize, Step, and Stop words
import nltk
import re
import pickle
import string  
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stops = stopwords.words("english")
stops.extend(['et','al','editor','review','abstract','result','conslusion','spacing','href','html','http','title', 'stats', 'bio'])


#stemmer = PorterStemmer()
stemmer = SnowballStemmer('english')

print('nltk version: ' + nltk.__version__)

def rm_munnezz(tk):
    return [w for w in tk if (w not in string.punctuation) and (not re.match(r'\d+',w)) and (not re.match(r'^[^a-zA-Z]',w))]

def uscore_intra_sym(tk):
    return [re.sub('[^a-zA-Z\d\s]','_',w) for w in tk]

def stem(tw):
    return [stemmer.stem(word) for word in tw]

def stop(sl):
    return [w for w in sl if not w in stops]


tokens = df['txt'].apply(nltk.word_tokenize)
token_words = tokens.apply(rm_munnezz)
token_words = token_words.apply(uscore_intra_sym)
stemmed_list = token_words.apply(stem)
meaningful_words = stemmed_list.apply(stop)

#meaningful_words = token_words.apply(stop)

df['processed'] = meaningful_words.apply(" ".join)

# save the processed corpus
f = open('results/docs.pckl', 'wb')
pickle.dump(df, f)
f.close()
df.shape

In [ ]:
# detect duplicated with cosine similarity
import numpy as np
import scipy.cluster
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import linkage
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy

f = open('results/docs.pckl', 'rb')
df = pickle.load(f)
f.close()

vectoriser = TfidfVectorizer(max_df=0.2,min_df=2)
doc_train = vectoriser.fit_transform(df['processed'])
print(df.shape)

# remove docs with no significant terms
w=np.max(doc_train,axis=1)
w=w.toarray()[:,0]
df = df[w!=0]

doc_train = vectoriser.fit_transform(df['processed'])
print(df.shape)

TD = pairwise_distances(doc_train,metric='cosine',n_jobs=-1)
np.fill_diagonal(TD,0)
TD = squareform(TD)
Z = linkage(TD,method='single')


ct = hierarchy.cut_tree(Z, height=0.1)
ct = ct.reshape(-1)
uct = np.unique(ct)
w = np.array([np.sum(ct==i) for i in uct])

dropidx = np.array([],int)
for ci in uct[w==2]:
    k = np.where((ct==ci) & (df['source'].values!='pubmed'))[0]
    dropidx = np.append(dropidx,k)
dropidx

retainidx = [i for i in list(range(df.shape[0])) if i not in dropidx]
print(len(retainidx))
print(df.shape[0] - len(dropidx))
df = df.iloc[retainidx,]

# save the processed corpus
f = open('results/docs.pckl', 'wb')
pickle.dump(df, f)
f.close()



## Topic model hyperparameters optimization

LDA hyperparameter optimization include the search for an optimal number of topics T. A grid search has been performed within a rage of values. Perplexity has been adopted as a measure of fitness to choose the optimal number of topics. Alpha and Beta parameters have been left to 1/T, recognized as optimal value.

In [ ]:
# Search for optimal number of topics
import pickle
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
print('sklearn version: ' + sklearn.__version__)


f = open('results/docs.pckl', 'rb')
df = pickle.load(f)
f.close()

vectoriser = CountVectorizer(max_df=0.2,min_df=2,
                             #max_features=10000,
                             ngram_range=(1,1))
doc_train = vectoriser.fit_transform(df['processed'])

perplexity = []
score = []
tt = []
topics = (5,10,20,22,24,26,28,29,30,32,34,36,38,40,45,50,55,60,70,80,90,100)
for T in topics:
    print(T)
    # Fit LDA to the data 
    LDA = LatentDirichletAllocation(n_components = T, 
                                    learning_method='batch',
                                    verbose=0, random_state=0,
                                    n_jobs=25)
    lda = LDA.fit(doc_train)
    perplexity.append(lda.perplexity(doc_train))
    score.append(lda.score(doc_train))
    tt.append(T)
    

df_scores = pd.DataFrame({'T':tt,'score':score,'perplexity':perplexity})

# save results on file
f = open('results/topic_scores.pckl', 'wb')
pickle.dump(df_scores, f)
f.close()



In [ ]:
# plot perplexity vs number of topics
import matplotlib.pyplot as plt
import numpy as np
import pickle


f = open('results/topic_scores.pckl', 'rb')
df_scores = pickle.load(f)
f.close()

df_scores.plot(kind='line',x='T',y='perplexity',grid=True,figsize=(10,5))
   

## Learn topics from the corpus



In [3]:
# Compute of words frequencies

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
import numpy as np
import pickle

f = open('results/docs.pckl', 'rb')
df = pickle.load(f)
f.close()

vectorizer = CountVectorizer(max_df=0.2,min_df=2,ngram_range=(1,1))
X = vectorizer.fit_transform(df['processed'])

print(X.shape) # check shape of the document-term matrix

terms_count = X.sum(axis=0)
terms_count = terms_count / float(terms_count.max())
dict_word_frequency = {}

terms = vectorizer.get_feature_names()
for i in range(len(terms)):
    dict_word_frequency[terms[i]] = terms_count[0,i]    


wcloud = WordCloud(background_color="white",mask=None, max_words=1000,\
                        max_font_size=60,min_font_size=10,prefer_horizontal=0.9,
                        contour_width=3,contour_color='black')
wcloud.generate_from_frequencies(dict_word_frequency)

plt.figure(figsize=(50,20))
plt.imshow(wcloud, interpolation='bilinear')
plt.title('Corpus top words',fontsize=50)
plt.show()



(27570, 18449)


<Figure size 5000x2000 with 1 Axes>

In [4]:
import pickle


# get the optimal number of topics from grid search results
f = open('results/topic_scores.pckl', 'rb')
df_scores = pickle.load(f)
f.close()
n_topics = df_scores['T'].values[df_scores['perplexity'].idxmin()]
print('Optimal number of topics = %d' % n_topics)
model = LatentDirichletAllocation(n_components=n_topics, 
                                    learning_method='batch',
                                    verbose=0, random_state=0,
                                    n_jobs=25)
topic_matrix = model.fit_transform(X)

print(topic_matrix.shape)

# save document topic distributions
f = open('results/topic_matrix.pckl', 'wb')
pickle.dump(topic_matrix, f)
f.close()

Optimal number of topics = 36
(27570, 36)


In [ ]:
# show topic top word clouds 
import math   
import matplotlib.pyplot as plt
from wordcloud import WordCloud 
terms_count = 25
numtopics = len(model.components_)

terms = vectorizer.get_feature_names()

ncol=4
nrow = int(math.ceil(numtopics/ncol))
fig, ax = plt.subplots(nrows=nrow, ncols=ncol,figsize=(25, 30))
fig.suptitle('Topics identified in COVID-19 literature', fontsize=30)

#Looping over lda components to get topics and their related terms with high probabilities
for idx,topic in enumerate(model.components_):    
    #print('Topic# ',idx)
    abs_topic = abs(topic)
    topic_terms = [[terms[i],topic[i]] for i in abs_topic.argsort()[:-terms_count-1:-1]]
    topic_terms_sorted = [[terms[i], topic[i]] for i in abs_topic.argsort()[:-terms_count - 1:-1]]
    topic_words = []
    for i in range(terms_count):
        topic_words.append(topic_terms_sorted[i][0])
    #print(','.join( word for word in topic_words))
    #print("")
    dict_word_frequency = {}
    
    for i in range(terms_count):
        dict_word_frequency[topic_terms_sorted[i][0]] = topic_terms_sorted[i][1]    
    wcloud = WordCloud(background_color="white",mask=None, max_words=100,\
                        max_font_size=60,min_font_size=10,prefer_horizontal=0.9,
                        contour_width=3,contour_color='black')
    wcloud.generate_from_frequencies(dict_word_frequency)
    i = idx // ncol
    j = idx % ncol
    ax[i,j].imshow(wcloud, interpolation='bilinear')
    ax[i,j].set_axis_off()
    ax[i,j].set_title('#%d' % idx,fontsize=20)
fig.savefig("Topics-topwords.pdf")
  


In [5]:
import pickle

f = open('results/docs.pckl', 'rb')
df = pickle.load(f)
f.close()

f = open('results/topic_matrix.pckl', 'rb')
topic_matrix = pickle.load(f)
f.close()

terms = vectorizer.get_feature_names()

def topic_top_words(model, feature_names, n_top_words):
    tword=[]
    for topic_idx, topic in enumerate(model.components_):
        #message = "Topic #%d: " % topic_idx
        message = ",".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(message)
    return(tword)

# Define helper functions
def doc_topic(topic_matrix):
    keys = topic_matrix.argmax(axis=1).tolist()
    return keys

def doc_topic2(topic_matrix):
    keys = topic_matrix.argsort(axis=1)
    return keys[:,-2]


ttop = topic_top_words(model, terms, 10)
topic = [ttop[i] for i in doc_topic(topic_matrix)]

topic2 = [ttop[i] for i in doc_topic2(topic_matrix)]


df['First topic']=topic
df['Second topic']=topic2
df['First topic id']=doc_topic(topic_matrix)
df['Second topic id']=doc_topic2(topic_matrix)

# entropy of each document
e = np.sum(-topic_matrix * np.log2(topic_matrix),axis=1)

df['entropy']=e

In [6]:
# categorie francesca
#f = open('results/docs.pckl', 'rb')
#df = pickle.load(f)
#f.close()

frdois = ['10.1101/2020.05.12.091256','10.3390/jcm9040982']
cls = ['network']*2
frdois.extend(['10.1080/07391102.2020.1756411','10.1038/d41586-020-01221-y','10.1016/j.compbiomed.2020.103670'])
cls.extend(['vaccine_serological']*3)
frdois.extend(['10.1038/s41591-020-0944-y','10.21203/rs.3.rs-27220/v1','10.1101/2020.03.15.20033472','10.1101/2020.05.05.079194','10.1101/2020.05.05.079095','10.1038/s41421-020-0168-9','10.1101/2020.05.18.101717','10.1101/2020.05.18.100545','10.1016/j.cell.2020.05.025','10.15252/embj.20105114','10.1038/s41591-020-0901-9','10.1101/2020.05.06.081695','10.1101/2020.04.08.029769','10.1038/s41586-020-2456-9','10.1101/2020.05.20.20107813','10.1101/2020.05.20.106294','10.1016/j.cell.2020.04.026','10.1080/22221751.2020.1747363','10.1101/2020.06.22.165225','10.1101/2020.04.21.051912','10.1016/j.stem.2020.06.015','10.1101/2020.05.25.115600','10.1126/science.abc1669','10.1016/j.cell.2020.05.006','10.1101/2020.04.29.20084327','10.1016/j.cell.2020.04.004','10.1101/2020.05.02.20084673','10.1101/2020.05.06.081695','10.2139/ssrn.3564998'])
cls.extend(['sequencing']*29)
frdois.extend(['10.1101/2020.05.21.108506','10.1101/2020.04.09.034942','10.1101/2020.03.30.016790','10.1016/j.meegid.2020.104351'])
cls.extend(['phylo']*4)
frdois.extend(['10.13140/RG.2.2.20588.10886','10.1093/bib/bbw051','10.1093/bib/bbw051','10.1101/2020.03.11.986836','10.1016/j.gendis.2020.04.002'])
cls.extend(['drug_docking']*5)

print(len(cls))
frcls = dict(zip(frdois,cls))

t = [frcls.get(i,'NA') for i in df['doi']]
df['annotation'] = t


43


In [108]:
# select document with low entropy and those with first topic belonging to biomarker related

# topics related to biomarkers
#ourtopicids = [33,3,4,27,23,20]
ourtopicids = [0,1,33,27,20]


from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

print(np.sum(df['annotation']!='NA'))

t=df[df['entropy']<(-0.5*np.log2(1/n_topics))]

t = t[t['First topic id'].isin(ourtopicids)] # | t['Second topic id'].isin(ourtopicids)]
t.drop('txt', axis=1, inplace=True)
t.drop('processed', axis=1, inplace=True)
t['doi link'] = '=HYPERLINK("https://doi.org/'+t['doi'] + '","'+t['doi']+'")'
t.to_excel('docs-filtered.xlsx',index=False)

# shows the table with the number of docs in each topic category
t.groupby(['First topic','source']).count()['doi']


34


First topic                                                               source 
antibodi,igg,serolog,igm,assay,respons,test,convalesc,plasma,neutral      biorxiv     23
                                                                          medrxiv     94
                                                                          pubmed     140
cell,ace2,express,receptor,lung,immun,human,gene,enzym,may                biorxiv    151
                                                                          medrxiv     96
                                                                          pubmed     480
drug,virus,vaccin,genom,viral,treatment,sequenc,effect,potenti,therapeut  biorxiv    194
                                                                          medrxiv     42
                                                                          pubmed     777
predict,model,learn,score,deep,valid,perform,method,machin,imag           biorxiv      1
                            

In [109]:
# annotate references with journal names, categories, and scimago ranking percentile
from crossref.restful import Works
import pandas as pd

sjr = pd.read_csv('data/scimagojr 2019.csv',sep=';')


works = Works()

jj = []
jc = []
jsi = []

for index, row in t.iterrows():
    w = works.doi(row['doi'])
    if w:
        if 'container-title' in w:
            #print(w['container-title'])
            if len(w['container-title'])>0:
                jj.append(w['container-title'][0])
                cat = sjr[sjr['Title'].str.match(w['container-title'][0])]
                if cat.shape[0]>0:
                    jc.append(cat['Categories'].values[0])
                    jsi.append(cat['SJR'].values[0])
                else:
                    jc.append('NA')
                    jsi.append(0)
            else:
                jj.append('NA')
                jc.append('NA')
                jsi.append(0)
        else:
            jj.append('NA')
            jc.append('NA')
            jsi.append(0)
    else:
        jj.append('NA')
        jc.append('NA')
        jsi.append(0)

t['Journal'] = jj
t['Journal Categories'] = jc
t['Journal SJR'] = jsi
t['Journal SJR'] = t['Journal SJR'].str.replace(',','.')
t['Journal SJR'] = t['Journal SJR'].astype(float)

t.to_excel('docs-filtered2.xlsx',index=False)

ValueError: could not convert string to float: '0,245'

In [110]:
t['Journal SJR']

40            0
45            0
66            0
68        0,245
83            0
128           0
137       1,601
144       1,361
147           0
148       1,158
149       1,158
172      36,691
176       6,141
186       2,406
221       0,755
249       2,994
252       1,608
254       1,119
261       1,633
262       0,694
269       2,163
275       7,920
276       1,649
289       0,971
291       3,280
300       3,410
318       1,185
324           0
334           0
343       1,608
          ...  
28834         0
28835         0
28837         0
28841         0
28842         0
28843         0
28853         0
28857         0
28858         0
28859         0
28871         0
28876         0
28878         0
28879         0
28881         0
28882         0
28889         0
28893         0
28895         0
28896         0
28897         0
28898         0
28899         0
28900         0
28903         0
28905         0
28906         0
28916         0
28922         0
28930         0
Name: Journal SJR, Lengt

In [111]:
t['Journal SJR'] = t['Journal SJR'].str.replace(',','.')


In [112]:
t['Journal SJR']

40          NaN
45          NaN
66          NaN
68        0.245
83          NaN
128         NaN
137       1.601
144       1.361
147         NaN
148       1.158
149       1.158
172      36.691
176       6.141
186       2.406
221       0.755
249       2.994
252       1.608
254       1.119
261       1.633
262       0.694
269       2.163
275       7.920
276       1.649
289       0.971
291       3.280
300       3.410
318       1.185
324         NaN
334         NaN
343       1.608
          ...  
28834       NaN
28835       NaN
28837       NaN
28841       NaN
28842       NaN
28843       NaN
28853       NaN
28857       NaN
28858       NaN
28859       NaN
28871       NaN
28876       NaN
28878       NaN
28879       NaN
28881       NaN
28882       NaN
28889       NaN
28893       NaN
28895       NaN
28896       NaN
28897       NaN
28898       NaN
28899       NaN
28900       NaN
28903       NaN
28905       NaN
28906       NaN
28916       NaN
28922       NaN
28930       NaN
Name: Journal SJR, Lengt

In [114]:
t['Journal SJR']



40          NaN
45          NaN
66          NaN
68        0.245
83          NaN
128         NaN
137       1.601
144       1.361
147         NaN
148       1.158
149       1.158
172      36.691
176       6.141
186       2.406
221       0.755
249       2.994
252       1.608
254       1.119
261       1.633
262       0.694
269       2.163
275       7.920
276       1.649
289       0.971
291       3.280
300       3.410
318       1.185
324         NaN
334         NaN
343       1.608
          ...  
28834       NaN
28835       NaN
28837       NaN
28841       NaN
28842       NaN
28843       NaN
28853       NaN
28857       NaN
28858       NaN
28859       NaN
28871       NaN
28876       NaN
28878       NaN
28879       NaN
28881       NaN
28882       NaN
28889       NaN
28893       NaN
28895       NaN
28896       NaN
28897       NaN
28898       NaN
28899       NaN
28900       NaN
28903       NaN
28905       NaN
28906       NaN
28916       NaN
28922       NaN
28930       NaN
Name: Journal SJR, Lengt

In [115]:
t.to_excel('docs-filtered2.xlsx',index=False)

In [120]:
print(t['entropy'].median(),0.5*np.log2(1/36))

1.8653424648136232 -2.584962500721156


In [130]:
df[df['title'].str.match('.*drug repurposing.*')]

,year,doi,title,txt,source,tlen,processed,First topic,Second topic,First topic id,Second topic id,entropy,annotation
1106,2020,10.1016/j.lfs.2020.117963,"Virtual screening, ADME/Tox predictions and th...","virtual screening, adme/tox predictions and th...",pubmed,122,virtual screen adme_tox predict drug repurpos ...,"protein,bind,spike,structur,human,viral,sars_c...","drug,virus,vaccin,genom,viral,treatment,sequen...",20,27,1.170804,NA
6381,2020,10.1080/07391102.2020.1775127,Identification of potential inhibitors of SARS...,identification of potential inhibitors of sars...,pubmed,165,identif potenti inhibitor sars_cov_2 endoribon...,"protein,bind,spike,structur,human,viral,sars_c...","drug,virus,vaccin,genom,viral,treatment,sequen...",20,27,1.327902,NA
8721,2020,10.1111/febs.15369,Understanding SARS-CoV-2 endocytosis for COVID...,understanding sars-cov-2 endocytosis for covid...,pubmed,67,understand sars_cov_2 endocytosi covid_19 drug...,"drug,virus,vaccin,genom,viral,treatment,sequen...","cell,ace2,express,receptor,lung,immun,human,ge...",27,33,1.665919,NA
11590,2020,10.1096/fba.2020-00015,The role of growth factor receptors in viral i...,the role of growth factor receptors in viral i...,pubmed,142,role growth factor receptor viral infect oppor...,"drug,virus,vaccin,genom,viral,treatment,sequen...","cell,ace2,express,receptor,lung,immun,human,ge...",27,33,2.199779,NA
14772,2020,10.1038/s41586-020-2286-9,A SARS-CoV-2 protein interaction map reveals t...,a sars-cov-2 protein interaction map reveals t...,pubmed,74,sars_cov_2 protein interact map reveal target ...,"drug,virus,vaccin,genom,viral,treatment,sequen...","protein,bind,spike,structur,human,viral,sars_c...",27,20,1.569275,NA
17791,2020,10.1016/j.ijantimicag.2020.105984,Combating devastating COVID-19 by drug repurpo...,combating devastating covid-19 by drug repurpo...,pubmed,51,combat devast covid_19 drug repurpos despit ad...,"drug,virus,vaccin,genom,viral,treatment,sequen...","trial,systemat,search,treatment,evid,includ,hy...",27,9,1.274839,NA
17904,2020,10.1016/j.drudis.2020.04.005,Boosting the arsenal against COVID-19 through ...,boosting the arsenal against covid-19 through ...,pubmed,77,boost arsenal covid_19 comput drug repurpos,"drug,virus,vaccin,genom,viral,treatment,sequen...","research,articl,right,reserv,impact,protect,co...",27,24,2.521803,NA
20057,2020,10.1080/07391102.2020.1753577,Targeting SARS-CoV-2: a systematic drug repurp...,targeting sars-cov-2: a systematic drug repurp...,pubmed,155,target sars_cov_2 systemat drug repurpos appro...,"protein,bind,spike,structur,human,viral,sars_c...","drug,virus,vaccin,genom,viral,treatment,sequen...",20,27,1.481687,NA
20436,2020,10.1080/07391102.2020.1752802,Computational studies of drug repurposing and ...,computational studies of drug repurposing and ...,pubmed,146,comput studi drug repurpos synerg lopinavir os...,"protein,bind,spike,structur,human,viral,sars_c...","day,group,treatment,ventil,hospit,oxygen,treat...",20,10,1.912257,NA
21798,2020,10.1038/d41587-020-00003-1,Coronavirus puts drug repurposing on the fast ...,coronavirus puts drug repurposing on the fast ...,pubmed,52,coronavirus put drug repurpos fast track,"drug,virus,vaccin,genom,viral,treatment,sequen...","countri,outbreak,public,itali,respons,emerg,wo...",27,5,2.235374,NA
